In [27]:
from collections import deque


class VictoryFlag(Exception):
    pass


class Dude:
    node = None
    enemy = None
    friends = None
    hp = None
    power = None

    def __init__(self, node):
        assert(node.dude is None)
        self.node = node
        node.dude = self
        self.friends.add(self)
        self.hp = 200
        self.power = 3

    def move(self):
        if not self.enemy.friends:
            raise VictoryFlag()

        # Don't move if next to enemy
        if self._near_enemy(self.node):
            return
        
        next_node = self._find_move()
        if next_node:
            next_node.dude = self
            self.node.dude = None
            self.node = next_node

    def attack(self):
        enemies = [n.dude for n in self.node.links if isinstance(n.dude, self.enemy)]
        if not enemies:
            return
        target = sorted(enemies, key=lambda e: (e.hp, e.node.pos))[0]
        target.hp -= self.power
        if target.hp < 1:
            target.friends.remove(target)
            target.node.dude = None

    @classmethod
    def _near_enemy(cls, node):
        return any(isinstance(n.dude, cls.enemy) for n in node.links)

    def _find_move(self):
        # Breadth-first search to find an empty space near an enemy
        seen = {self.node}
        fringe = deque([(None, self.node)])
        while fringe:
            prev_start, node = fringe.popleft()
            for next_node in node.links:
                start = prev_start or next_node
                if next_node.dude or next_node in seen:
                    continue
                if self._near_enemy(next_node):
                    return start
                seen.add(next_node)
                fringe.append((start, next_node))
        return None


class Goblin(Dude):
    friends = None
    enemy = None

    
class Elf(Dude):
    friends = None
    enemy = None

    
class Node:
    dude = None
    links = None
    pos = None
    
    def __init__(self, row, col, top_node, left_node):
        self.links = []
        self.pos = (row, col)
        if top_node:
            self.links.append(top_node)
            top_node.links.append(self)
        if left_node:
            self.links.append(left_node)
            left_node.links.append(self)
            
            
class Board:
    turns = None
    nodes = {}

    def __init__(self, filename):
        self.turns = 0
        Goblin.friends = set()
        Elf.friends = set()
        Goblin.enemy = Elf
        Elf.enemy = Goblin

        parse_node = {
            '.': (Node, None),
            '#': (None, None),
            'G': (Node, Goblin),
            'E': (Node, Elf),
        }

        with open(filename) as infile:
            row = None
            for r, l in enumerate(infile):
                prev_row = row
                row = []
                for c, rune in enumerate(l.strip()):
                    node = None
                    node_class, dude_class = parse_node[rune]
                    if node_class:
                        node = node_class(r, c, prev_row[c], row[-1])
                        self.nodes[(r,c)] = node
                    if dude_class:
                        dude_class(node)
                    row.append(node)
      
    def tick(self):
        dudes = Elf.friends | Goblin.friends
        for d in sorted(dudes, key=lambda d: d.node.pos):
            if d.hp < 1:
                continue
            try:
                d.move()
            except VictoryFlag:
                return False
            d.attack()
        self.turns += 1
        return True
    
    def run(self):
        conflict = True
        while conflict:
            print(self.turns, len(Goblin.friends), len(Elf.friends))
            conflict = self.tick()
        print(self.turns * (sum(d.hp for d in (Elf.friends | Goblin.friends))))

In [29]:
b = Board('input.txt')
b.run()

0 20 10
1 20 10
2 20 10
3 20 10
4 20 10
5 20 10
6 20 10
7 20 10
8 20 10
9 20 10
10 20 10
11 20 10
12 20 10
13 20 10
14 20 10
15 20 10
16 20 10
17 20 10
18 20 10
19 20 10
20 20 10
21 20 10
22 20 10
23 20 10
24 20 10
25 20 10
26 20 8
27 20 8
28 20 8
29 20 8
30 20 8
31 20 8
32 20 8
33 20 8
34 20 8
35 20 8
36 19 8
37 19 8
38 19 8
39 19 8
40 19 8
41 19 7
42 19 7
43 19 7
44 19 7
45 19 7
46 19 7
47 19 7
48 19 7
49 19 7
50 19 7
51 19 7
52 19 7
53 19 7
54 19 6
55 19 6
56 19 6
57 19 6
58 19 6
59 19 5
60 19 4
61 19 4
62 19 4
63 19 4
64 19 4
65 18 4
66 18 4
67 18 4
68 18 3
69 18 3
70 17 3
71 17 2
72 17 2
73 17 1
74 17 1
75 17 1
76 17 1
77 17 1
78 17 1
79 17 1
80 17 1
81 17 1
82 17 1
83 17 1
84 17 1
85 17 1
86 17 1
87 17 1
88 17 1
89 17 1
90 17 1
91 17 1
92 17 1
93 17 1
94 17 1
95 17 1
96 17 1
97 17 1
98 17 1
99 17 1
100 17 1
101 17 1
102 17 1
103 17 1
104 17 1
105 17 1
269430


In [7]:
def debug():
    print(b.turns)
    for d in sorted(Goblin.friends | Elf.friends, key=lambda d: d.node.pos):
        print(type(d).__name__[0], d.node.pos, d.hp)